In [2]:
# documentation for the API
# https://stackapi.readthedocs.io/en/latest/

!pip3 install stackapi

In [ ]:
##import required libraries

import time
import datetime
import pandas as pd
from stackapi import StackAPI # import the API wrapper
from stackapi import StackAPIError # import error methods for debugging

# connect to the stackoverflow API using python wrapper
def stackoverflow_connect():
  try:
    # create a SITE object to query upon
    SITE = StackAPI('stackoverflow')
    SITE.page_size = 100
    SITE.max_pages=50 # this number would have to be tweaked
  except StackAPIError as e:
    print(e.message)
  return SITE

# get the json object for the current month 
def fetch_questions_JSON(SITE):
  # month_begin = int(datetime.datetime.now().replace(day = 1).timestamp())   # get the timestamp of the beginning of current month based on when the query is run  
  now = int(datetime.datetime.now().timestamp())   # get the timestamp of the current times based on when the query is run
  month_begin = int((datetime.datetime.now() - datetime.timedelta(30)).timestamp()) # use a timedelta to get the data 30 days ago from current date

  # get a JSON load of all the question in a timeframe ** 
  questions = SITE.fetch('questions' , fromdate = month_begin , todate = now
                        #  , min = 20
                         , tagged = 'julia'
                        , sort = 'votes')
  return questions 

# read the json file into a pandas DF
def json_df_redux(questions):
  # pull the items from the JSON questions field 
  df = pd.json_normalize(questions['items'])
  return df


# upload the raw df into database using spark (intentionally left blank)
def load_raw_df_to_db(df):
  # conver the pandas df into spark df 
  raw_spark_df = spark.createDataFrame(df)
  mode = "overwrite"
  url = "jdbc:redshift://localhost:5432/etl_pipeline"
  properties = {"user": "<username>",
                "password": "<password>",
                "driver": "org.redshift.Driver"
                }
  raw_spark_df.write.jdbc(url=url,
                table = "raw_stackoverflow_questions",
                mode = mode,
                properties = properties)

# lets transform the questions pandas df to make some more insights about the tags

def tag_enrichment(df):
  df1 = df.explode('tags') # we are explosing the tags column into rows to get each tag detail
  monthly_top_20_tags = df1['tags'].value_counts()[:20]
  monthly_top_20_tags = monthly_top_20_tags.to_frame()
  return monthly_top_20_tags


if __name__ == "__main__":
    SITE = stackoverflow_connect()
    questions = fetch_questions_JSON(SITE)
    df = json_df_redux(questions)
    # load_raw_df_to_db(df) # optional step to push the raw dataframe into database of choice
    top_tags = tag_enrichment(df)
    email_tag_data(top_tags)


Error: unable to send email


Test the functions locally

In [10]:
from stackapi import StackAPI # import the API wrapper
import datetime


# create a SITE object to query upon
SITE = StackAPI('stackoverflow')
# SITE.page_size = 100
# SITE.max_pages=100 # this number would have to be tweaked

# now = int(datetime.datetime.now().timestamp())  
# month_begin = int((datetime.datetime.now() - datetime.timedelta(9999)).timestamp())

# get a JSON load of all the question in a timeframe ** 
questions = SITE.fetch('questions' 
                    # , fromdate = month_begin , todate = now
                    # , min = 20
                     , tagged = 'julia'
                    , sort = 'votes')

In [ ]:
import pandas as pd
df = pd.json_normalize(questions['items'])
df